In [2]:
import pandas as pd
import ipaddress
import numpy as np
import pandasql as ps

In [18]:
filename = 'log20170630.csv'
#parsing_dates didn't work.
#couldn't use between times. 

In [23]:
def pipeline(filename):
    df = pd.read_csv(filename,nrows=20000)
    df['ip']= to_ipaddress(df['ip'])
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = [date.date() for date in df['date']]
    return df[['ip', 'date','cik','accession']]

def to_ipaddress(arr): 
    arr = [np.char.replace(ip, ip[-3:],'000') for ip in arr]
    return [int(ipaddress.IPv4Address(x)) for x in arr]

def setupiptable(filename):
    df = pd.read_csv(filename)
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: int(ipaddress.IPv4Address(x)))
    df.iloc[:,1] = df.iloc[:,1].apply(lambda x: int(ipaddress.IPv4Address(x)))
    df.columns = ["from_ip","to_ip","code","country"]
    return df.drop(columns=['code'])

In [20]:
df1 = pipeline(filename)


In [21]:
dfgeo = setupiptable('~/Downloads/IP2LOCATION-LITE-DB1.CSV')
dfgeo.head(1)

,from_ip,to_ip,country
0,16777216,16777471,United States of America


In [22]:
sqlcode = '''
    SELECT * 
    FROM df, dfgeo
    WHERE ip BETWEEN from_ip AND to_ip
'''
newdf = ps.sqldf(sqlcode, locals()).drop(columns=['from_ip','to_ip'])

In [24]:
newdf['country'].value_counts()

United States of America                                35026
China                                                    4541
United Kingdom of Great Britain and Northern Ireland     4215
Canada                                                    975
Czechia                                                   841
India                                                     545
Italy                                                     520
Ukraine                                                   488
Bulgaria                                                  473
Netherlands                                               450
Hong Kong                                                 413
Russian Federation                                        319
Australia                                                 239
Spain                                                     191
Greece                                                    185
Korea (Republic of)                                       169
Brazil  

In [25]:
sqlcode_topcik = '''
    SELECT cik, accession, country, COUNT(*)
    FROM newdf
    GROUP BY cik
    ORDER BY 4 DESC
    LIMIT 20;
'''
#topcikdf= ps.sqldf(sqlcode_topcik, locals())

In [ ]:
sqlcode_topcikbycountry = '''
    SELECT cik, accession, country, COUNT(*)
    FROM newdf
    GROUP BY cik
    ORDER BY 4 DESC
    LIMIT 20;
'''
#topcikbycountrydf= ps.sqldf(sqlcode_topcikbycountry, locals())

In [2]:
sqlcode_topdoc = '''
    SELECT accession, country, COUNT(*)
    FROM newdf
    GROUP BY accession
    ORDER BY 3 DESC
    LIMIT 20;
'''
#topdocdf = ps.sqldf(sqlcode_topdoc, locals())

In [ ]:
sqlcode_topdocbycountry = '''
    SELECT COUNT(accession), country
    FROM newdf
    GROUP BY country
    ORDER BY 1 DESC
    LIMIT 20;
'''
#topdocbycountrydf= ps.sqldf(sqlcode_topdocbycountry, locals())